# Week 9 Class 2 NLP Furniture Demand, Example 8.5 

In [ ]:
import pyomo.environ as pe
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Maximize profit where profit is price minus cost:

$𝑧=(𝑝_S−60) 𝑥_S+(𝑝_T−45) 𝑥_T$

$𝑝_𝑆=220−0.4𝑥_𝑆$

$𝑝_𝑇=180−0.2𝑥_𝑇$

subject to

$2𝑥_S+𝑥_T≤500$     (inspection) 

$2𝑥_S+3𝑥_T≤800$       (assembly)
     
We can also constrain $𝑥_S,𝑥_T$ to be integers.

In [ ]:
model = pe.ConcreteModel()

# Decision variables
DV_indexes = ['Sofa', 'Table']
model.x = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
                 
# Objective function (𝑝1−60)𝑥1+(𝑝2−45)𝑥2
model.obj = pe.Objective(expr = ((220 - 0.4*model.x['Sofa']) - 60) * model.x['Sofa'] +
                         ((180 - 0.2*model.x['Table']) - 45) * model.x['Table'],
                         sense = pe.maximize)

# Constraints
# inspection 2𝑥_S+𝑥_T≤500
model.cons_inspect = pe.Constraint(expr = 2*model.x['Sofa'] + model.x['Table'] <= 500)

# assembly 2𝑥_S+3𝑥_T≤800
model.cons_assem = pe.Constraint(expr = 2*model.x['Sofa'] + 3*model.x['Table'] <= 800)

model.pprint()

In [ ]:
opt = pe.SolverFactory('ipopt')
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

In [ ]:
print('The Optimal Values')
obj_val = model.obj.expr()
print(f'optimal objective value = {obj_val:.2f}')

In [ ]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for c in DV:
        DV_solution.loc[DV.name,c] = round(DV[c].value,1)
DV_solution

## Sensitivity Analysis
Let's do some sensitivity analysis by varying the RHS of the assembly constraint from 750 to 860 and see the effects on the Decision Variables and the objective function.

In [ ]:
def run_model(rhs):
    
    model = pe.ConcreteModel()

    # Decision variables
    DV_indexes = ['Sofa', 'Table']
    model.x = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
                 
    # Objective function (𝑝1−60)𝑥1+(𝑝2−45)𝑥2
    model.obj = pe.Objective(expr = ((220 - 0.4*model.x['Sofa']) - 60) * model.x['Sofa'] +
                             ((180 - 0.2*model.x['Table']) - 45) * model.x['Table'],
                             sense = pe.maximize)

    # Constraints
    # inspection 2𝑥_S+𝑥_T≤500
    model.cons_inspect = pe.Constraint(expr = 2*model.x['Sofa'] + model.x['Table'] <= 500)
    
    # assembly 2𝑥_S+3𝑥_T≤800
    model.cons_assem = pe.Constraint(expr = 2*model.x['Sofa'] + 3*model.x['Table'] <= rhs)

    opt.solve(model)
    return model

In [ ]:
rng = np.arange(750, 870, 10)

In [ ]:
solution = pd.DataFrame()
for con in rng:
    model = run_model(con)
    for DV in model.component_objects(pe.Var):
         for var in DV:
            solution.loc[con, var] = round(DV[var].value, 1)
    solution.loc[con,'opt_profit'] = round(model.obj.expr(), 2)
    # if con != 750:
    #     solution.loc[con, 'Obj Change/Unit'] = (solution.loc[con, 'opt_profit']- 
    #                                             solution.loc[con - 10,'opt_profit'])/10

In [ ]:
solution

In [ ]:
solution['RHS'] = solution.index

In [ ]:
df = pd.melt(solution, 'RHS')
df

In [ ]:
plt.figure(figsize = (8,5))
sns.lineplot(x = 'RHS', y = 'value', hue = 'variable', data = df[df['variable'] != 'opt_profit'])
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize = (8,5))
sns.lineplot(x = 'RHS', y = 'value', data = df[df['variable'] == 'opt_profit'])
plt.ylabel('profit')
plt.grid()
plt.show()